In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("user_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475688 entries, 0 to 475687
Data columns (total 31 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Gender                                     475688 non-null  object 
 1   First Name                                 475688 non-null  object 
 2   Last Name                                  475688 non-null  object 
 3   City                                       475688 non-null  object 
 4   State                                      475688 non-null  object 
 5   Country                                    475688 non-null  object 
 6   Age                                        475688 non-null  int64  
 7   Annual Vacation Days                       475688 non-null  float64
 8   Average Monthly Spend on Entertainment     475688 non-null  float64
 9   Number of Online Purchases in Last Month   475688 non-null  float64
 10  Number o

In [3]:
df = df.drop(["First Name" , "Last Name" , "City" , "State", "Country" , "Education Level"] , axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475688 entries, 0 to 475687
Data columns (total 25 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Gender                                     475688 non-null  object 
 1   Age                                        475688 non-null  int64  
 2   Annual Vacation Days                       475688 non-null  float64
 3   Average Monthly Spend on Entertainment     475688 non-null  float64
 4   Number of Online Purchases in Last Month   475688 non-null  float64
 5   Number of Charity Donations in Last Year   475688 non-null  float64
 6   Average Weekly Exercise Hours              475688 non-null  float64
 7   Investment Portfolio Value                 475688 non-null  float64
 8   Health Consciousness Rating                475688 non-null  float64
 9   Average Daily Screen Time                  475688 non-null  float64
 10  Environm

In [4]:
# One hot encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in df.columns :
  if df[column].dtype == object :
    df[column] = le.fit_transform(df[column])
    
df

,Gender,Age,Annual Vacation Days,Average Monthly Spend on Entertainment,Number of Online Purchases in Last Month,Number of Charity Donations in Last Year,Average Weekly Exercise Hours,Investment Portfolio Value,Health Consciousness Rating,Average Daily Screen Time,...,Financial Wellness Index,Lifestyle Balance Score,Entertainment Engagement Factor,Social Responsibility Index,Work-Life Balance Indicator,Investment Risk Appetite,Eco-Consciousness Metric,Stress Management Score,Time Management Skill,Lifestyle Choice
0,1,49,30.02,0.00,166.0,3.0,0.00,295.63,0.68,5.86,...,100.00,36.49,2.26,0.0,0.49,0.45,0.0,4.93,44.37,3
1,1,43,23.64,7.23,130.0,0.0,2.80,0.00,0.64,3.52,...,100.00,25.72,0.89,0.0,0.54,3.66,0.0,0.78,1.09,0
2,0,37,7.91,8.09,107.0,0.0,0.00,0.00,0.75,6.74,...,70.42,7.14,1.27,0.0,0.00,7.09,0.0,1.16,33.64,11
3,1,22,20.63,6.39,138.0,0.0,0.00,0.00,0.57,5.38,...,103.17,27.14,1.79,0.0,0.00,1.61,0.0,3.06,100.00,1
4,1,38,19.47,7.80,115.0,0.0,0.07,11.50,5.35,6.00,...,100.00,23.31,1.42,0.0,0.27,1.47,0.0,4.07,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475683,1,34,0.88,43.53,100.0,0.0,0.24,11.12,0.00,3.71,...,100.00,0.98,2.20,0.0,0.77,0.73,0.0,2.90,1.11,4
475684,0,24,8.04,42.33,78.0,0.0,0.00,10.16,0.71,1.67,...,100.00,1.03,2.84,0.0,0.00,6.41,0.0,5.98,7.12,2
475685,0,30,21.44,24.24,189.0,0.0,0.00,0.00,0.88,1.59,...,100.00,14.94,1.07,0.0,0.00,0.01,0.0,0.84,3.62,10
475686,0,62,27.87,45.78,115.0,0.0,0.00,1.61,0.77,7.35,...,0.00,32.25,2.08,0.0,0.32,2.04,0.0,4.27,6.10,6


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score
import numpy as np

# โหลดข้อมูล

X = df.drop(["Lifestyle Choice"],axis=1)
y = df["Lifestyle Choice"]

# สร้างข้อมูล semi-supervised
# rng = np.random.RandomState(42)
# mask_unlabeled = rng.rand(len(y)) < 0.5
# y[mask_unlabeled] = -1

# # แบ่งข้อมูล
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # ใช้ LabelPropagation
# lp_model = LabelPropagation()
# lp_model.fit(X_train, y_train)

# # ทำนาย
# y_pred = lp_model.predict(X_test)

# # ประเมินความแม่นยำ
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

/var/folders/3d/cszn3fy1733cnwsqbxysmdnr0000gn/T/ipykernel_60429/8729388.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[mask_unlabeled] = -1


: 